## Part 2: Vice Presidential and Presidential Debate Analysis
* Debates are a tool for the candidates to appeal to an audience deciding who to vote for in the election. 
    * Viewers of these debates might already have a candidate in mind, but the audience isn't directed towards only one side, like the conventions. 
* This analysis will include a classifier, which will try to predict if a spoken line is republican or democratic.
    * Goal: find key words used by each candidate to further their arguments. Also, find accuracy of the classifier for my hypothesis. 
* I will also be making a model based off of token length to see if speaking a lot is a persuasion choice made by the candidates. 

In [1]:
import pandas as pd

In [2]:
presdebate = pd.read_csv("/Users/emmatarcson/Documents/data_science/RhetoricalFactor-analysis/data/welkerPRESdebate.csv")
presdebate = presdebate[['Aff','transcript']]
presdebate

,Aff,transcript
0,R,How are you doing? How are you?
1,R,"So as you know, 2.2 million people modeled out..."
2,R,There was a very big spike in Texas. It’s now ...
3,R,"I can tell you from personal experience, I was..."
4,D,"220,000 Americans dead. You hear nothing else ..."
...,...,...
319,R,"Before the plague came in, just before, I was ..."
320,R,Success is going to bring us together. We are ...
321,D,"I will say, I’m an American President. I repre..."
322,D,"We can grow this economy, we can deal with the..."


In [3]:
vpdebate = pd.read_csv("/Users/emmatarcson/Documents/data_science/RhetoricalFactor-analysis/data/pageVPdebate.csv")
vpdebate = vpdebate[['Aff','transcript']]
vpdebate

,Aff,transcript
0,D,"Thank you, Susan. Well, the American people ha..."
1,D,"Can you imagine if you knew on January 28th, a..."
2,D,… right to reelection based on this.
3,R,"Susan, thank you. And I want to thank the Comm..."
4,R,And I believe it saved hundreds of thousands o...
...,...,...
202,R,I look at the relationship between Justice Rut...
203,D,"First of all, I love hearing from our young le..."
204,D,"And brings me to Joe, Joe Biden. One of the re..."
205,D,Joe has a longstanding reputation of working a...
